# Build the demo. 

Use `testutils` to help with it. The actions are annotated here to try to provide some of the issuesstates = Path('states') # Should be LOCAL

try:
    shutil.rmtree(states)
except OSError:
    pass
states.mkdir(parents=True,exist_ok=False)
(states / '.ignore').touch()

In [ ]:
from pathlib import Path
import shutil
import json
import hashlib

states = Path('states').resolve() # Will need to the absollute later since we will move around

try:
    shutil.rmtree(states)
except OSError:
    pass
states.mkdir(parents=True,exist_ok=False)
(states / '.ignore').touch()

In [ ]:
def hashsrc(num):
    """Will quickly hash all of the files for validation later"""
    state = {}
    for dirpath, dirnames, filenames in os.walk('src'):
        for file in filenames:
            full = os.path.join(dirpath,file)
            with open(full,'rb') as f:
                sha1 = hashlib.sha1(f.read()).hexdigest()
            rel = os.path.relpath(full,'src')
            state[rel] = sha1
    with open(states / f'{num:02d}.json','wt') as f:
        json.dump(state,f)

In [ ]:
import sys,os


PWD0 = os.getcwd()
print(f'{PWD0 = }')

if (abspath := os.path.abspath('../tests/')) not in sys.path:
    sys.path.insert(0,abspath)
import testutils

In [ ]:
test = testutils.Tester(name='restore_poc')
print(f'{test.pwd = }')

Set up the config. Also include hashes but don't use them.

In [ ]:
test.config["renames"] = "mtime"
test.config['compare'] = "mtime"
test.config['dt'] = 0.1
test.config['get_hashes'] = True # Won't always use but I want to have
test.config["hash_type"] = "sha1"
test.write_config()

## State 0

Start building the initial view. As part of the proof-of-concept, this will *not* be what we restore. We will restore to **State 1** 

In [ ]:
test.write_pre('src/never_change.txt','will never change')

# Moves
test.write_pre('src/will_move_at_1.txt','will move at step 1')
test.write_pre('src/will_move_at_1_mod_at_2.txt','will move at step 1 then mod')

test.write_pre('src/will_move_at_3_del_at_4.txt','will move at 3 and del at 4')

test.write_pre('src/subdir/will/change/file_wont_at_3.txt','Will move to a new subdir')
test.write_pre('src/subdir/will/change/file_will_at_3.txt','Will move to a new subdir and change the name')

test.write_pre('src/will_move_a_and_b_at_2-a.txt','will move at 2')
shutil.copy2('src/will_move_a_and_b_at_2-a.txt','src/will_move_a_and_b_at_2-b.txt')

test.write_pre('src/will_move_a_only_at_3-a.txt','will move at 3')
shutil.copy2('src/will_move_a_only_at_3-a.txt','src/will_move_a_only_at_3-b.txt')

# Deletes
test.write_pre('src/delete_at_3.txt','will be deleted at 3')
test.write_pre('src/delete_at_2_restore_at_4','restore at 4 (but change data)')

# Modified
test.write_pre('src/will_mod_at_all','mod all')
test.write_pre('src/will_mod_at_2','mod 2')


test.cli("--init", "config.py")
hashsrc(0)

## State 1

In [ ]:
shutil.move('src/will_move_at_1.txt','src/moved_at_1.txt')
shutil.move('src/will_move_at_1_mod_at_2.txt','src/moved_at_1_mod_at_2.txt')

test.write_post('src/will_mod_at_all','mod all1')

test.write_pre('src/new_at_1.txt','new 1')

test.cli("config.py")
hashsrc(1)

## State 2

In [ ]:
test.write_post('src/moved_at_1_mod_at_2.txt','at 2',mode='at')

shutil.move('src/will_move_a_and_b_at_2-a.txt','src/moved_a_and_b_at_2-a.txt')
shutil.move('src/will_move_a_and_b_at_2-b.txt','src/moved_a_and_b_at_2-b.txt')

os.unlink('src/delete_at_2_restore_at_4')

test.write_post('src/will_mod_at_all','mod all12')
test.write_post('src/will_mod_at_2','mod 2_DONE')

test.write_pre('src/new_at_2.txt','new 2')

test.cli("config.py")
hashsrc(2)

## State 3

In [ ]:
shutil.move('src/will_move_a_only_at_3-a.txt','src/moved_a_only_at_3-a.txt')
shutil.move('src/will_move_at_3_del_at_4.txt','src/did_move_at_3_del_at_4.txt')

os.makedirs('src/subdir/did/change/')
shutil.move('src/subdir/will/change/file_wont_at_3.txt','src/subdir/did/change/file_wont_at_3.txt')
shutil.move('src/subdir/will/change/file_will_at_3.txt','src/subdir/did/change/file_did_at_3.txt')

test.write_post('src/will_mod_at_all','mod all123')

os.unlink('src/delete_at_3.txt')


test.cli("config.py")
hashsrc(3)

## State 4 (final)

In [ ]:
test.write_post('src/delete_at_2_restore_at_4','restore at 4 (but CHANGED data)')
test.write_post('src/will_mod_at_all','mod all1234')

test.write_pre('src/new_at_4.txt','new 4')

os.unlink('src/did_move_at_3_del_at_4.txt')

test.cli('config.py')
hashsrc(4)